# NBA Immaculate Grid Application

This program imports NBA box score data from the SportsDataVerse to create an "immaculate grid" application, a game where two NBA franchises are selected at random and the user is prompted to input the name of a player who has played at least one game for each of the teams between the 2014 and 2024 NBA seasons. After the user inputs their selection, the program will indicate whether their selection was correct, as well as print a list of all eligible players for the category.

#### Importing libraries:

In [1]:
import pandas as pd
import numpy as np
import random 
import sportsdataverse as sdv

pd.set_option('display.max_columns', 100)

#### Accessing NBA data through sportsdataverse:

In [2]:
nba_data = sdv.nba.load_nba_player_boxscore(seasons = range(2014, 2025), return_as_pandas = True)

100%|██████████| 11/11 [00:07<00:00,  1.51it/s]


#### Performing some initial data inspection:

In [3]:
nba_data.head()

,game_id,season,season_type,game_date,game_date_time,athlete_id,athlete_display_name,team_id,team_name,team_location,team_short_display_name,minutes,field_goals_made,field_goals_attempted,three_point_field_goals_made,three_point_field_goals_attempted,free_throws_made,free_throws_attempted,offensive_rebounds,defensive_rebounds,rebounds,assists,steals,blocks,turnovers,fouls,plus_minus,points,starter,ejected,did_not_play,reason,active,athlete_jersey,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_display_name,team_uid,team_slug,team_logo,team_abbreviation,team_color,team_alternate_color,home_away,team_winner,team_score,opponent_team_id,opponent_team_name,opponent_team_location,opponent_team_display_name,opponent_team_abbreviation,opponent_team_logo,opponent_team_color,opponent_team_alternate_color,opponent_team_score
0,400559378,2014,3,2014-06-15,2014-06-15 20:00:00-04:00,469,Rashard Lewis,14,Heat,Miami,Heat,9.0,1,2,1,2,0,0,0,2,2,1,0,0,0,2,+8,3,True,False,False,COACH'S DECISION,False,9,R. Lewis,https://a.espncdn.com/i/headshots/nba/players/...,Power Forward,PF,Miami Heat,s:40~l:46~t:14,miami-heat,https://a.espncdn.com/i/teamlogos/nba/500/mia.png,MIA,000000,000000,away,False,87,24,Spurs,San Antonio,San Antonio Spurs,SA,https://a.espncdn.com/i/teamlogos/nba/500/sa.png,000000,061922,104
1,400559378,2014,3,2014-06-15,2014-06-15 20:00:00-04:00,1977,Chris Bosh,14,Heat,Miami,Heat,39.0,6,14,0,5,1,2,0,7,7,2,1,0,0,2,-20,13,True,False,False,COACH'S DECISION,False,1,C. Bosh,https://a.espncdn.com/i/headshots/nba/players/...,Power Forward,PF,Miami Heat,s:40~l:46~t:14,miami-heat,https://a.espncdn.com/i/teamlogos/nba/500/mia.png,MIA,000000,000000,away,False,87,24,Spurs,San Antonio,San Antonio Spurs,SA,https://a.espncdn.com/i/teamlogos/nba/500/sa.png,000000,061922,104
2,400559378,2014,3,2014-06-15,2014-06-15 20:00:00-04:00,1966,LeBron James,14,Heat,Miami,Heat,41.0,10,21,3,9,8,9,1,9,10,5,0,2,1,1,-16,31,True,False,False,COACH'S DECISION,False,6,L. James,https://a.espncdn.com/i/headshots/nba/players/...,Small Forward,SF,Miami Heat,s:40~l:46~t:14,miami-heat,https://a.espncdn.com/i/teamlogos/nba/500/mia.png,MIA,000000,000000,away,False,87,24,Spurs,San Antonio,San Antonio Spurs,SA,https://a.espncdn.com/i/teamlogos/nba/500/sa.png,000000,061922,104
3,400559378,2014,3,2014-06-15,2014-06-15 20:00:00-04:00,9,Ray Allen,14,Heat,Miami,Heat,31.0,1,8,1,3,2,2,1,4,5,2,1,0,4,2,-18,5,True,False,False,COACH'S DECISION,False,34,R. Allen,https://a.espncdn.com/i/headshots/nba/players/...,Shooting Guard,SG,Miami Heat,s:40~l:46~t:14,miami-heat,https://a.espncdn.com/i/teamlogos/nba/500/mia.png,MIA,000000,000000,away,False,87,24,Spurs,San Antonio,San Antonio Spurs,SA,https://a.espncdn.com/i/teamlogos/nba/500/sa.png,000000,061922,104
4,400559378,2014,3,2014-06-15,2014-06-15 20:00:00-04:00,1987,Dwyane Wade,14,Heat,Miami,Heat,36.0,4,12,1,2,2,4,0,3,3,1,0,0,3,3,-10,11,True,False,False,COACH'S DECISION,False,3,D. Wade,https://a.espncdn.com/i/headshots/nba/players/...,Guard,G,Miami Heat,s:40~l:46~t:14,miami-heat,https://a.espncdn.com/i/teamlogos/nba/500/mia.png,MIA,000000,000000,away,False,87,24,Spurs,San Antonio,San Antonio Spurs,SA,https://a.espncdn.com/i/teamlogos/nba/500/sa.png,000000,061922,104


In [4]:
nba_data.shape

(348577, 57)

In [5]:
nba_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348577 entries, 0 to 348576
Data columns (total 57 columns):
 #   Column                             Non-Null Count   Dtype                                      
---  ------                             --------------   -----                                      
 0   game_id                            348577 non-null  int32[pyarrow]                             
 1   season                             348577 non-null  int32[pyarrow]                             
 2   season_type                        348577 non-null  int32[pyarrow]                             
 3   game_date                          348577 non-null  date32[day][pyarrow]                       
 4   game_date_time                     348577 non-null  timestamp[us, tz=America/New_York][pyarrow]
 5   athlete_id                         348577 non-null  int32[pyarrow]                             
 6   athlete_display_name               348577 non-null  large_string[pyarrow]   

In [6]:
nba_data['team_display_name'].unique()

<ArrowExtensionArray>
[            'Miami Heat',      'San Antonio Spurs',  'Oklahoma City Thunder',
         'Indiana Pacers',            'LA Clippers',     'Washington Wizards',
 'Portland Trail Blazers',          'Brooklyn Nets',       'Dallas Mavericks',
        'Toronto Raptors',  'Golden State Warriors',      'Memphis Grizzlies',
          'Atlanta Hawks',        'Houston Rockets',          'Chicago Bulls',
      'Charlotte Bobcats',           'Phoenix Suns',       'Sacramento Kings',
         'Denver Nuggets',        'Milwaukee Bucks',              'Utah Jazz',
 'Minnesota Timberwolves',   'New Orleans Pelicans',        'Detroit Pistons',
     'Los Angeles Lakers',         'Boston Celtics',    'Cleveland Cavaliers',
     'Philadelphia 76ers',        'New York Knicks',          'Orlando Magic',
 'Eastern Conf All-Stars', 'Western Conf All-Stars',      'Charlotte Hornets',
            'Team LeBron',           'Team Stephen',                  'World',
                    'USA',    

#### Cleaning the DataFrame:

In [7]:
# Renaming some columns
nba_data = nba_data.rename(columns = {'athlete_id':'id', 'athlete_display_name':'player', 'team_display_name':'team'})

# Merging both Charlotte teams
nba_data['team'] = np.where(nba_data['team'] == 'Charlotte Bobcats', 'Charlotte Hornets', nba_data['team'])

# Defining a list of ineligible teams
ineligible_teams = ['Team LeBron', 'Eastern Conf All-Stars', 'Western Conf All-Stars', 'Team Giannis', 'World', 
                    'Team Durant', 'USA', 'Leb', 'Team Stephen', 'Usa']

# Creating our condition: if team is in "ineligible", remove it
condition = [True if not (x in ineligible_teams) else False for x in nba_data['team']]

# Applying our condition to the DataFrame
nba_data = nba_data[condition].reset_index(drop = True)

# Only keep necessary columns
nba_data = nba_data[['id', 'player', 'team']]

In [8]:
nba_data.head()

,id,player,team
0,469,Rashard Lewis,Miami Heat
1,1977,Chris Bosh,Miami Heat
2,1966,LeBron James,Miami Heat
3,9,Ray Allen,Miami Heat
4,1987,Dwyane Wade,Miami Heat


#### Aggregating the data:

In [9]:
# Performing the groupby
nba_grid = nba_data.groupby(['id', 'player'], as_index = False)['team'].unique()

In [12]:
nba_grid.head()

,id,player,team
0,9,Ray Allen,[Miami Heat]
1,25,Metta World Peace,"[New York Knicks, Los Angeles Lakers]"
2,63,Chauncey Billups,[Detroit Pistons]
3,91,Elton Brand,"[Atlanta Hawks, Philadelphia 76ers]"
4,110,Kobe Bryant,[Los Angeles Lakers]


#### Checking validity of aggregations:

In [13]:
list(nba_grid[nba_grid['player'] == 'LeBron James']['team'])

[array(['Miami Heat', 'Cleveland Cavaliers', 'Los Angeles Lakers'],
       dtype=object)]

In [14]:
list(nba_grid[nba_grid['player'] == 'James Harden']['team'])

[array(['Houston Rockets', 'Brooklyn Nets', 'Philadelphia 76ers',
        'LA Clippers'], dtype=object)]

In [15]:
list(nba_grid[nba_grid['player'] == 'Chris Paul']['team'])

[array(['LA Clippers', 'Houston Rockets', 'Oklahoma City Thunder',
        'Phoenix Suns', 'Golden State Warriors'], dtype=object)]

#### Creating immaculate grid application:

In [16]:
def Grid():
    
    # Defining official teams list
    teams = ['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls', 
             'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons', 'Golden State Warriors', 
             'Houston Rockets', 'Indiana Pacers', 'LA Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 
             'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks', 
             'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 
             'Sacramento Kings', 'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
    
    # Randomly selecting two teams for our grid
    selected_teams = random.sample(teams, 2)
    
    # Filtering DataFrame to only include players who played for both selected teams
    nba_grid_temp = nba_grid[[True if np.isin(selected_teams, x).all() else False for x in nba_grid['team']]]
    
    # Selecting all eligible players
    player_list = list(nba_grid_temp['player'])
    
    # Restarting if length of eligible players list is less than 3
    if (len(player_list) < 3):
        return(Grid())
    
    # Printing the selected teams
    print('Team #1:', selected_teams[0])
    print('Team #2:', selected_teams[1])
    
    # Getting user input
    player = input('\nEnter your player: ')
    
    # Checking if player is in the DataFrame
    if (player in player_list):
        print('\nCorrect!')
    else:
        print('\nIncorrect!')
    
    print('\nEligible players:\n', player_list)

#### Playing the game:

In [23]:
Grid()

Team #1: Philadelphia 76ers
Team #2: Memphis Grizzlies



Enter your player:  Danny Green



Correct!

Eligible players:
 ['Anthony Tolliver', 'Jerryd Bayless', 'Danny Green', 'Elliot Williams', 'Darius Morris', 'Seth Curry', 'Tim Frazier', 'James Ennis III', 'Dakota Mathias', "De'Anthony Melton", 'Kenneth Lofton Jr.']
